In [1]:
!pip install soynlp

     |████████████████████████████████| 419kB 29.9MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [125]:
import pandas as pd
import pickle
from pathlib import Path, PurePath
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
# from pykospacing import Spacing
from soynlp.normalizer import emoticon_normalize, repeat_normalize, remove_doublespace
import numpy as np

In [126]:
pp = Path('/content/gdrive/MyDrive/chat_bot/data')
file_list = [i.name for i in pp.glob('*.pkl')]
print(file_list)

ff = ( pickle.load(open(pp / f, 'rb')) for f in file_list )
df = pd.concat(ff, axis=0, ignore_index=True)
df.tail()

['df_강남구_세곡동.pkl', 'df_강남구_개포동.pkl', 'df_강남구_자곡동.pkl', 'df_송파구_송파1동.pkl', 'df_송파구_송파2동.pkl', 'df_송파구_석촌동.pkl', 'df_송파구_문정1동.pkl', 'df_송파구_문정2동.pkl', 'df_송파구_오륜동.pkl', 'df_송파구_오금동.pkl', 'df_종로구_사직동.pkl', 'df_종로구_삼청동.pkl', 'df_종로구_부암동.pkl', 'df_강남구_청담동.pkl', 'df_종로구_평창동.pkl', 'df_강남구_삼성동.pkl', 'df_종로구_무악동.pkl', 'df_강남구_논현동.pkl', 'df_종로구_이화동.pkl', 'df_종로구_창신1동.pkl', 'df_종로구_창신2동.pkl', 'df_종로구_혜화동.pkl', 'df_강남구_수서동.pkl', 'df_강남구_대치동.pkl', 'df_강남구_신사동.pkl']


,restaurant,review,answer,total,taste,quantity,delivery,location
606472,갈비&해물나라,...... ....,고객님의 첫 주문에 만족감을 드리지 못해 사과 드립니다. 하지만 다음에 주문 하실때...,5,3,4,5,강남구 신사동
606473,갈비&해물나라,양도 푸짐하니 많고 맛도 좋네요,맛있게 드시고 소중한 후기 남겨 주셔서 진심으로 감사드립니다. 언제나 고객님의 만족...,5,5,5,4,강남구 신사동
606474,갈비&해물나라,양도 많고 매콤하니 좋네요 술인주로 좋을듯,저희 닭도리탕은 압력솥으로 요리하기 때문에 고기가 연하고 부드럽습니다. 저희 음식점...,5,4,5,3,강남구 신사동
606475,갈비&해물나라,양도 많고 좋아요. 밥도 챙겨주시고 콜라도 주시구요. 50분 걸린다는 배달이 한시간...,주문이 밀리다 보니 고객님께 불편을 끼쳐드려 사과 드립니다. 저희 업소는 언제나 고...,5,,,,강남구 신사동
606476,반월식당,배달도 빨라서 좋았는데 맛도 좋습니다! 덕분에 든든한 저녁 잘먹었습니다~~~!,맛있는 리뷰 감사합니다 (❁ᴗ͈ˬᴗ͈)⁾⁾⁾ 매일 신선한 재료와 빠른 배달로 만족드...,5,5,5,5,강남구 신사동


In [127]:
# 이모티콘 제거
emoji_pattern = re.compile("[" 
    u"\U0001F600-\U0001F64F" # emoticons 
    u"\U0001F300-\U0001F5FF" # symbols & pictographs 
    u"\U0001F680-\U0001F6FF" # transport & map symbols 
    u"\U0001F1E0-\U0001F1FF" # flags (iOS) 
    u"\U00010000-\U0010FFFF"
         "]+", flags=re.UNICODE) 

# 특수문자, 등 제거
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", 
                 "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", 
                 "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
                 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
                 
def clean_punc(text, punct, mapping):
  
  text = re.sub(r'[A-Za-z\\**]+[님]', '', text) # user id 언급 삭제
  for p in mapping: 
    text = text.replace(p, mapping[p]) 

  # special case & extra emoji
  specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': '', 
              '❣':'', '❤':'','๑˃̵ᴗ˂̵':'','☆':'', '♥':'','⭐':'','❤️':''} 
  for s in specials: 
    text = text.replace(s, specials[s]) 
  
  # for p in punct: 
  #   text = text.replace(p, f' {p} ') 
  
  return text.strip()

def clean_text(texts): 
  # corpus = [] 
  # for i in range(0, len(texts)): 
  # review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',texts) #remove punctuation
  review = re.sub(r'[@%\\*=()/#&\+á?\xc3\xa1\-\|\:\-\_\$\'\"\^]', '',texts) 
  # review = re.sub(r'\d+','', texts)# remove number 
  review = review.lower() #lower case
  review = re.sub(r'\s+', ' ', review) #remove extra space 
  review = re.sub(r'<[^>]+>','',review) #remove Html tags 
  review = re.sub(r'\s+', ' ', review) #remove spaces 
  review = re.sub(r"^\s+", '', review) #remove space from start 
  review = re.sub(r'\s+$', '', review) #remove space from the end 
    # corpus.append(review) 
    
  return review


def pre_processing_text(text):
  if text is None:
    return None
  text = emoji_pattern.sub(r'',text)
  text = clean_text(text)
  text = emoticon_normalize(text, num_repeats=2)
  text = clean_punc(text, punct, punct_mapping)

  return text



In [128]:
# drop None
df = df[~df['answer'].isnull()]
df = df[~df['review'].isnull()]
df = df[~(df['answer']=='')]
df = df[~(df['review']=='')]
df = df[~(df['taste']=='')]
df = df[~(df['delivery']=='')]
print(df.isnull().any(), df.shape)


restaurant    False
review        False
answer        False
total         False
taste         False
quantity      False
delivery      False
location      False
dtype: bool (566743, 8)


In [129]:
df.shape

(566743, 8)

In [130]:
df['review_cleaned'] = df['review'].apply(pre_processing_text)
df['answer_cleaned'] = df['answer'].apply(pre_processing_text)

len(df.review_cleaned.to_list() + df.answer_cleaned.to_list())

1133486

In [140]:
df.head()

,Q,A,star_avg
12,와 진짜 대박 맛있어서 암청 많이 먹다가 리뷰하는거 까먹어서 지금 남기네요 ㄷㄷㄷ ...,맛있게 드셨다니 저희도 정말 기쁩니다 ㅎㅎ 더 맛있고 정성스럽게 조리할테니 치킨 생...,5.0
13,치킨이 양이 적어보이는데 치킨 덩어리가 엄청커요!! 목덕후인 저는 목이 없어 슬프지...,소중한 사진 리뷰 감사합니다 ! 앞으로도 더 맛있는 음식으로 찾아뵐게요 ~~ 좋은 ...,4.0
14,처음시켜 보는데 상담히 만족합니다. 배달 주문하기 힘드신 어머님이 부탁하셔서 시켜드...,첫 주문에 만족하셨다니 정말 다행이에요 ㅎㅎ 앞으로도 초심 잃지 않고 저희 모두 열...,5.0
15,맛있게 잘 먹었습니다,소중한 리뷰 감사합니다 ! 더 맛있고 정성스러운 음식으로 찾아뵐게요 좋은 하루 보내...,5.0
16,맛있었어요. 감사합니다.,맛있게 드셨다니 정말 다행이에요 감사합니다 ! 좋은 하루 보내시고 항상 건강하세요><,5.0


In [132]:
df[['taste','quantity','delivery']] = df[['taste','quantity','delivery']].apply(pd.to_numeric)
df['star_avg'] = df.loc[:,['taste','quantity','delivery']].mean(axis=1)
df['star_avg'] = df['star_avg'].apply(lambda x: round(x,1))

In [69]:
df.head()

,taste,quantity,delivery,review_cleaned,answer_cleaned,star_avg
12,5,5,5,와 진짜 대박 맛있어서 암청 많이 먹다가 리뷰하는거 까먹어서 지금 남기네요 ㄷㄷㄷ ...,맛있게 드셨다니 저희도 정말 기쁩니다 ㅎㅎ 더 맛있고 정성스럽게 조리할테니 치킨 생...,5.0
13,4,4,4,치킨이 양이 적어보이는데 치킨 덩어리가 엄청커요!! 목덕후인 저는 목이 없어 슬프지...,소중한 사진 리뷰 감사합니다 ! 앞으로도 더 맛있는 음식으로 찾아뵐게요 ~~ 좋은 ...,4.0
14,5,5,5,처음시켜 보는데 상담히 만족합니다. 배달 주문하기 힘드신 어머님이 부탁하셔서 시켜드...,첫 주문에 만족하셨다니 정말 다행이에요 ㅎㅎ 앞으로도 초심 잃지 않고 저희 모두 열...,5.0
15,5,5,5,맛있게 잘 먹었습니다,소중한 리뷰 감사합니다 ! 더 맛있고 정성스러운 음식으로 찾아뵐게요 좋은 하루 보내...,5.0
16,5,5,5,맛있었어요. 감사합니다.,맛있게 드셨다니 정말 다행이에요 감사합니다 ! 좋은 하루 보내시고 항상 건강하세요><,5.0


In [133]:
cols = [c for c in df.columns if c not in ['review_cleaned','answer_cleaned','star_avg']]
cols
df.drop(columns=cols, inplace=True)
df.rename({'review_cleaned':'Q','answer_cleaned':'A','atar_avg':'label'}, axis=1, inplace=True)

# preprocessing 과정에서 생기는 공백(아마도 정규표현식이나 이모티콘만 있는 리뷰일듯)
df = df[~ ( (df['A']=='') | ( df['Q']=='' ) ) ]

In [158]:
df_small = df.sample(100000)
df_small.shape

In [160]:
df_small.to_csv(pp / 'ChatbotData_small.csv', index=False)
df.to_csv(pp / 'ChatbotData.csv', index=False)